## Tensor Flow Problem Sheet

These problems relate to the [TensorFlow](https://www.tensorflow.org/) python library for pattern recognition. This notebook uses the [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris).

### 1.  Use Tensorflow to create a model.
Create a model that uses a flower's sepal width / length and petal width / length to predict the species of Iris.

In [10]:
'''
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import genfromtxt

iD = genfromtxt('data.csv', delimiter=',')
irisData = np.array([iD])
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    irisTF = tf.constant(irisData)
    print(irisTF.eval())

# Read data from CSV adapted from https://www.tensorflow.org/versions/r1.1/programmers_guide/reading_data
'''

# Adapted from: https://github.com/salmanahmad4u/keras-iris/blob/master/iris_nn.py

import csv
import numpy as np
import keras as kr

#print(kr.methods)

# Load the dataset from csv
iris = list(csv.reader(open('data.csv')))[1:]

# Add inputs (sepal length, sepal width, petal length, petal width) to numpy array, comma separated, to 4th col.
inputs  = np.array(iris)[:4].astype(np.float)

outputs = np.array(iris)[:4]
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True) # Convert outputs to ints
# Encode the category integers as binary categorical vairables.
outputs_cats = kr.utils.to_categorical(outputs_ints)


### 2. Split the data into training and testing sets.
Investigate the *best way to do this* - write some code to randomly separate data if desired. Reference relevant material.

In [15]:
inds = np.random.permutation(len(inputs))
train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = inputs[test_inds],  outputs_cats[test_inds]

print(inputs_train)
print(outputs_train)

[[ 4.9  3.   1.4  0.2]
 [ 5.   3.6  1.4  0.2]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


### 3. Train the model.
Train the model using the test (should be training?) set.

### 4. Test the model.
Test your model using the testing set. Calculate and display clearly the error rate.